## Setup & read in data

In [ ]:
import pandas as pd
import sklearn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt   

In [ ]:
#this is the data including imputed values for test results etc 
all_data = pd.read_csv('imputed_all_data.csv')
parse_dates=['admission_date_structured']
labels = pd.read_csv('disch_los_k_modes_6.csv')
combined_clean = pd.read_csv('combined_clean.csv')


In [ ]:
orig_data_and_labels_comorb = pd.merge(all_data, labels,
                           how='right', on='id')

In [ ]:
#remove comorbidities 
orig_data_and_labels = orig_data_and_labels_comorb.drop(columns=['morbidity_Diabetes','morbidity_COPD','morbidity_Hypertension','morbidity_Heartdisease','morbidity_Renaldisease','morbidity_Tumor','morbidity_Metabolicdisorders','morbidity_Respiratorydiseases'])

In [ ]:
orig_data_and_labels

,Unnamed: 0,id,Height,Weight,smoking_structured,age,blood_sugar_d1_min,blood_sugar_d1_max,blood_sugar_d3_min,blood_sugar_d3_max,...,AlbumintoGlobulinRatio,Male,Consciousness,Temperature,Respiratoryrate,Redcelldistributionwidth,SystolicBP,DiastolicBP,Lymphocyte(%),cluster
0,0,100251,155.00000,60.0,0,62.0,4.660000,4.660000,4.925083,5.956664,...,1.382490,False,1.0,36.6,20.0,12.1,134.500000,68.500000,23.384615,3
1,1,101019,151.00000,63.0,0,72.0,5.300000,5.300000,3.902008,4.507097,...,1.129046,False,1.0,36.3,23.0,13.6,135.000000,80.000000,32.424242,1
2,2,102696,169.00000,80.0,0,63.0,5.190000,5.190000,5.538094,7.640623,...,1.367413,True,1.0,36.6,19.0,13.4,129.000000,91.000000,32.266155,1
3,3,101588,161.00000,57.0,0,39.0,4.939551,4.895466,4.694930,4.862858,...,1.125873,False,1.0,36.6,22.0,12.5,122.000000,74.500000,25.101747,1
4,4,100358,164.75961,65.0,0,62.0,5.793085,6.349398,5.576803,5.064091,...,1.006937,True,1.0,37.5,24.0,11.9,122.500000,80.000000,15.538462,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2792,2792,101460,172.00000,72.0,0,49.0,4.340000,4.340000,5.230405,3.933495,...,1.560000,True,0.0,36.8,24.0,12.2,100.326822,66.389026,32.307692,1
2793,2793,100047,172.00000,75.0,0,45.0,4.091314,7.067830,4.062333,4.329303,...,1.263107,True,1.0,36.8,22.0,11.4,101.593761,78.039515,18.589744,1
2794,2794,103038,174.00000,72.0,0,56.0,4.330000,4.330000,5.489082,5.853361,...,1.460000,True,1.0,36.5,26.0,13.1,124.000000,76.000000,37.792496,3
2795,2795,100889,164.75961,80.0,0,74.0,4.040000,4.040000,4.339569,4.426759,...,1.150000,True,1.0,36.5,26.0,12.5,139.500000,100.500000,14.313725,1


### Setup: multiclass

In [ ]:
orig_data_and_labels.cluster.value_counts()

1    1374
3    1018
4     173
2      97
0      73
5      62
Name: cluster, dtype: int64

Remove the two largest clusters; these are the non-severe patients 

In [ ]:
num_to_remove = 2 
# Remove the largest n clusters to focus on interesting ones 

value_counts = orig_data_and_labels.cluster.value_counts()
value_counts_list = value_counts.index.to_list()
interesting_clusters = value_counts_list[num_to_remove:len(value_counts)]

data_and_labels = orig_data_and_labels.loc[orig_data_and_labels['cluster'].isin(interesting_clusters)]

In [ ]:
data_and_labels.cluster.value_counts()

4    173
2     97
0     73
5     62
Name: cluster, dtype: int64

In [ ]:
data_and_labels2 = data_and_labels.copy()
data_and_labels2['cluster'] = data_and_labels2['cluster'].apply(lambda x: + x)

In [ ]:
from sklearn import preprocessing, model_selection

def preprocess(all_data):
#     Split the dataset into features and labels (clusters) - so we can normalise the features but not the labels
    all_X = all_data.iloc[:,2:len(all_data.columns)-1]
    all_y = all_data['cluster']
    
    # normalise the data so we have unit variance and mean 0 using built-in preprocessing method in sklearn
    scaler = preprocessing.StandardScaler()
    all_X= pd.DataFrame(scaler.fit_transform(all_X),columns=all_X.columns)
    
#     reset the indexes otherwise it was breaking
    all_X= all_X.reset_index()
    all_y = all_y.reset_index()
    all_X = all_X.drop(columns=['index'])
    all_y = all_y.drop(columns=['index'])
    
    #prepare data for training a model by splitting into training and testing data 
    return (all_X, all_y)

In [ ]:
all_X, all_y = preprocess(data_and_labels2)

In [ ]:
data_and_labels2

,Unnamed: 0,id,Height,Weight,smoking_structured,age,blood_sugar_d1_min,blood_sugar_d1_max,blood_sugar_d3_min,blood_sugar_d3_max,...,AlbumintoGlobulinRatio,Male,Consciousness,Temperature,Respiratoryrate,Redcelldistributionwidth,SystolicBP,DiastolicBP,Lymphocyte(%),cluster
13,13,102072,175.00000,60.000000,0,74.0,3.610000,3.610000,4.707085,4.628041,...,1.353447,True,1.0,36.4,20.0,14.6,118.000000,74.000000,38.297872,4
17,17,101282,164.75961,65.029389,0,70.0,5.195489,4.721443,5.327653,4.032294,...,1.237522,True,1.0,38.0,24.0,12.2,139.500000,79.500000,14.920635,0
18,18,101287,160.00000,60.000000,0,58.0,4.773361,5.972382,4.966484,4.477538,...,1.088416,False,1.0,36.6,22.0,13.8,136.000000,91.000000,10.510204,2
30,30,100234,168.00000,58.000000,0,43.0,5.160000,5.160000,4.276654,7.233130,...,1.372451,False,0.0,36.3,22.0,13.3,121.000000,70.500000,23.714286,5
32,32,102787,156.00000,65.000000,0,81.0,4.600000,4.600000,3.986601,6.947844,...,1.331862,False,1.0,36.5,19.0,16.9,102.000000,65.000000,28.918493,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2750,2750,102532,170.00000,70.000000,0,72.0,4.560000,4.560000,5.100000,5.100000,...,1.380000,True,1.0,36.5,22.0,13.2,145.500000,80.500000,10.317460,4
2753,2753,101429,160.00000,61.000000,0,62.0,4.890000,4.890000,4.891793,5.946383,...,1.360000,False,0.0,36.9,24.0,13.0,101.364050,64.353563,32.321429,5
2762,2762,101767,177.00000,72.000000,0,66.0,5.610000,5.610000,4.840000,4.840000,...,0.810000,True,1.0,36.5,26.0,13.7,105.000000,62.000000,13.308824,5
2772,2772,102548,170.00000,75.000000,1,68.0,4.800000,4.800000,5.373095,6.411728,...,1.440000,True,1.0,36.2,27.0,13.1,118.676547,70.397796,32.315789,4


## Classification setup

In [ ]:
#Set the class names
class_names = [4,2,0,5]

### setting up performance metrics calculations

In [ ]:
from sklearn import metrics
#method to compute performance metrics 
def performance_metrics(y_true, y_pred):
    #create a confusion matrix from results 
    cnf_matrix  = metrics.confusion_matrix(y_true, y_pred, labels=class_names, sample_weight=None)
#     # average over all classes to find overall fp,fn,tp,tn
#     FP = np.sum(cnf_matrix, axis=0) - np.diag(cnf_matrix)
#     FN = np.sum(cnf_matrix, axis=1) - np.diag(cnf_matrix)
#     TP = np.diag(cnf_matrix)
#     TN = np.sum(cnf_matrix) - (FP + FN + TP)
        
    #find out if each predicted label is right 
    correct_labels = 0
    for i, true_label in enumerate(y_true):
        if (true_label == y_pred[i]):
            correct_labels += 1

    #calculate overall accuracy
#     acc = np.round(correct_labels/y_pred.shape[0],2)
        
    # tp rate is the same as recall 
#     recall = np.sum(TP)/(np.sum(TP)+np.sum(FN))
#     FP_rate = np.sum(FP)/(np.sum(FP)+np.sum(TN))  
#     precision = np.sum(TP)/(np.sum(TP)+np.sum(FP)) 
#     f_measure = (2*precision*recall)/(precision + recall) 

    metrics_dict = metrics.classification_report(y_true,y_pred,output_dict=True)
    avg_metrics = metrics_dict['macro avg']
    
    recall = avg_metrics['recall']
    precision = avg_metrics['precision']
    f_measure = avg_metrics['f1-score']
    acc = metrics_dict['accuracy']
    
    print(metrics.classification_report(y_true,y_pred))
    
    return (acc, recall, precision, f_measure,
            np.round(cnf_matrix/cnf_matrix.sum(axis=1), 2))


In [ ]:
import seaborn as sn

#method to plot the confusion matrix 
def plot_confusion_matrix(confusion_matrix, class_names):
    conf_matrix = pd.DataFrame(confusion_matrix, index = [i for i in class_names], columns = [i for i in class_names])
    plt.figure()
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(conf_matrix, annot=True, annot_kws={"size": 16}, cmap="Blues")
    plt.show()

## k-fold cross validation

Here we use k-fold cross validation to evaluate the effectiveness of our models, where each 'fold' results in a slightly different train/test split, allowing us to see how the model beaves with different data. We can compute various metrics about the model performance, and from these results, we can choose the most effective classification model.

In [ ]:
from sklearn import model_selection 
from sklearn import linear_model
from sklearn import ensemble

def cross_val(all_X, all_y, class_names, downsampling, upsampling, down_prop, up_prop, clf, printing):
    print('cross-validating...')
    k_val = 3
    #use 3 splits 
    k_fold = model_selection.KFold(n_splits=k_val)
    strat_k_fold = model_selection.StratifiedKFold(n_splits=k_val, shuffle=True)

    #create empty arrays to store the results from each fold
    #we can then average these to get the overall classifications and performance 
    accuracies = np.empty(k_val)
    tp_rates = np.empty(k_val)
    precisions = np.empty(k_val)
    f_measures = np.empty(k_val)

    count = 0

    #split training data into training and validation sets
    for train_indices, validation_indices in strat_k_fold.split(all_X, all_y):
    #     print(X_train.index)
        X_training = all_X.iloc[train_indices]
        y_training = all_y.iloc[train_indices]
        X_validate = all_X.iloc[validation_indices]
        y_validate = all_y.iloc[validation_indices]
        
        # upsample the training data in each fold, if upsample = true
        #with stratified k fold, better not to upsample, as it preserves percentage of samples for each class 
#         print('value counts: ', y_training.value_counts())

        if(downsampling):
#             print('downsampling...')
            X_training, y_training = downsample(X_training, y_training, class_names, down_prop)
            if (printing):
                print('value counts: ', y_training.value_counts())

        if (upsampling):
#             print('upsampling...')
            X_training, y_training = upsample(X_training, y_training, class_names, up_prop)
            if (printing):
                print('value counts:', y_training.value_counts())
            
        # fit classifier
        clf.fit(X_training, y_training.values.ravel())
        
        #predict 
        y_predicted = clf.predict(X_validate)
        y_true_val = y_validate.cluster
        
        #if we are using the decision tree classifier we may want to export the tree to look at it 
        from sklearn import tree

        export_tree = False
        if(export_tree):
            for tree_in_forest in clf.estimators_:
                if(export_tree):
                    tree.export_graphviz(tree_in_forest, out_file="rotated_tree_eg.dot",filled = True, 
                                         feature_names = X_training.columns, class_names=class_names,
                                         rotate = True)
        if (printing):
            print(np.unique(y_predicted, return_counts=True))

        #see how validation set performs 
        acc, recall, precision, f_measure, confusion_matrix = performance_metrics(y_true_val.values, y_predicted)

        if (printing):
            #print performance results for each fold
            print("accuracy: ", acc)
            print("true positive rate / recall: ", recall)
            print("precision: ", precision)
            print("f measure: ", f_measure)
            plot_confusion_matrix(confusion_matrix, class_names)
            print("----------")

        # store info for each fold 
        accuracies[count] = acc
        tp_rates[count] = recall
        precisions[count] = precision
        f_measures[count] = f_measure
        count+=1
        
        
    #average scores
    cv_acc = np.mean(accuracies)
    cv_tp = np.mean(tp_rates)
    cv_precision = np.mean(precisions)
    cv_f_measure = np.mean(f_measures)


    print("overall average performance:")
    print("accuracy: ", cv_acc)
    print("true positive rate/recall: ", cv_tp)
    print("precision: ", cv_precision)
    print("f measure: ", cv_f_measure)
    
    return X_training, y_training, X_validate, y_validate, y_predicted, y_true_val, cv_f_measure


Method to perform grid search to find the best parameters for the random forest classifier, uses cross-val


Had to implement this manually due to library constraints and difficulties with upsampling properly inside gridsearch

In [ ]:
from sklearn import model_selection, pipeline, ensemble

def grid_search_rand_forest(data_X, data_y,class_names, downsampling, upsampling, down_prop, up_prop):

    best_f = 0.0
    
    # gridsearch for parameter optimisation
    param_grid = { 
        'n_estimators': [5, 10, 15, 20, 30, 40, 60, 80, 100],
        'max_depth': [5, 10, 15, 20, 25, None],
        'max_features': ['auto', 'sqrt', 'log2'],
        'min_samples_split': [2, 5, 10, 15, 20],
        'criterion' :['gini', 'entropy']}
    
    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for max_features in param_grid['max_features']:
                for min_samples_split in param_grid['min_samples_split']:
                    for criterion in param_grid['criterion']:
                        clf = ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, 
                                               min_samples_split=min_samples_split, criterion=criterion)

                        X_training, y_training, X_validate, y_validate, y_predicted, y_true_val, f_measure = cross_val(data_X, data_y, class_names, downsampling, upsampling, down_prop, up_prop, clf, printing=False)

                        if(f_measure > best_f):
                            best_f = f_measure
                            best_clf = clf
    
    # Create grid search object
#     clf = model_selection.GridSearchCV(estimator=ensemble.RandomForestClassifier(), param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

#     best_clf = clf.fit(data_X, data_y.values.ravel())

#     gridsearch_results = pd.DataFrame(best_clf.cv_results_)

    #sort according to overall rank score to find best params
#     best_params_found = gridsearch_results.sort_values('rank_test_score').iloc[0]
    print(best_clf)
    
    return best_clf

## Rebalancing classes

### Downsampling

To address class imbalance by removing some examples in most numerous classes

In [ ]:
def downsample(train_X, train_y, class_names, down_prop):
    
    train_data_and_labels = train_X.copy()
    train_data_and_labels['cluster'] = train_y
    
    #find the largest class and the number of samples in training
    value_counts = train_data_and_labels.cluster.value_counts()
    max_val = max(value_counts)

    most_numerous =  train_data_and_labels.cluster.value_counts().index[0]
    big_cluster = train_data_and_labels[train_data_and_labels['cluster']==most_numerous]
    new_train = train_data_and_labels[train_data_and_labels.cluster!=most_numerous]
    #remove more individuals in largest class if limited classes, as class imbalance is bigger here
    reduced_cluster = big_cluster.sample(replace=False, n=int(len(big_cluster)*down_prop), random_state=1)
    train_down = new_train.append(reduced_cluster)
    y_train_down = train_down['cluster']
    X_train_down = train_down.iloc[:,0:len(train_down.columns)-1]        
    
    return X_train_down, y_train_down

### Upsampling 

This improves the prediction accuracy for the classes containing fewer individuals.

In [ ]:
#upsample the least numerous classes
def upsample(train_X, train_y, class_names, up_prop):
    
    train_data_and_labels = train_X.copy()
    train_data_and_labels['cluster'] = train_y

    #find the largest class and the number of samples in training
    value_counts = train_data_and_labels.cluster.value_counts()
    max_val = max(value_counts)
        
    #upsampling all the classes in training data 
    for cluster in class_names:
        cluster_vals = train_data_and_labels[train_data_and_labels.cluster==int(cluster)]
        num_in_cluster = len(cluster_vals.index)
        num_extra_samples = int(up_prop*(max_val - num_in_cluster))
        new_samples = cluster_vals.sample(replace=True, n=num_extra_samples, random_state=1)
        train_data_and_labels = train_data_and_labels.append(new_samples)
            
#     print(train_data_and_labels.cluster.value_counts())
    
    X_train_up = train_data_and_labels.iloc[:,0:len(train_data_and_labels.columns)-1]
    y_train_up = train_data_and_labels['cluster']
    
    return X_train_up, y_train_up

Set parameters

In [ ]:
#need to do gridsearch 
# clf = grid_search_rand_forest(all_X, all_y, class_names, downsampling=True, upsampling=True, down_prop=1, up_prop=0.8)
#for multiclass kmeans
clf = ensemble.RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_split=10,
                       n_estimators=60)
#for binary kmeans

We now have a new dataframe containing a more equal distribution of each class to work with. 

Now we perform the k-fold cross validation on data. Since our dataset is small, the predictions can vary a lot based on which data is used for training/testing, which is why k-fold cross-val is useful to get overall prediction metrics. 
If upsampling is true then the least numerous classes will be upsampled.

#### Multiclass:

In [ ]:
X_training, y_training, X_validate, y_validate, y_predicted, y_true_val, cv_f_measure = cross_val(all_X, all_y, class_names, downsampling=True, upsampling=True, down_prop=1, up_prop=0.8, clf=clf, printing=False)

cross-validating...
              precision    recall  f1-score   support

           0       0.42      0.21      0.28        24
           2       0.53      0.76      0.62        33
           4       0.52      0.62      0.57        58
           5       0.43      0.15      0.22        20

    accuracy                           0.51       135
   macro avg       0.47      0.43      0.42       135
weighted avg       0.49      0.51      0.48       135

              precision    recall  f1-score   support

           0       0.40      0.33      0.36        24
           2       0.85      0.91      0.88        32
           4       0.65      0.67      0.66        58
           5       0.19      0.19      0.19        21

    accuracy                           0.59       135
   macro avg       0.52      0.53      0.52       135
weighted avg       0.58      0.59      0.59       135

              precision    recall  f1-score   support

           0       0.47      0.32      0.38        25
 